# Test monetization workflow

In [48]:
import pandas as pd
import brightway2 as bw
import xarray as xr
import numpy as np

As a first simple example, let's look at unit processes of solar and wind electricity generation. First, find the respective ecoinvent activities.

In [3]:
bw.projects.set_current("test")

In [35]:
eidb = bw.Database("ecoinvent_remind_default_2020")

PVactivities = [act for act in eidb if "electricity production, photovoltaic" == act["name"] and "DE" in act["location"]]

In [37]:
actPV = PVactivities[0]

In [38]:
WINDactivities = [act for act in eidb if "wind" in act["name"] and "DE" in act["location"]]
WINDactivities

['electricity production, wind, >3MW turbine, onshore' (kilowatt hour, DE, None),
 'electricity production, wind, <1MW turbine, onshore' (kilowatt hour, DE, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, DE, None),
 'electricity production, wind, 1-3MW turbine, offshore' (kilowatt hour, DE, None)]

In [39]:
actWind = WINDactivities[2]

Now we perform the LCIA calculation.

In [43]:
methods =  [
    ('ReCiPe Midpoint (H) V1.13', 'agricultural land occupation', 'ALOP'),
    ('ReCiPe Midpoint (H) V1.13', 'climate change', 'GWP100'),
    ('ReCiPe Midpoint (H) V1.13', 'fossil depletion', 'FDP'),
    ('ReCiPe Midpoint (H) V1.13', 'freshwater ecotoxicity', 'FETPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'freshwater eutrophication', 'FEP'),
    ('ReCiPe Midpoint (H) V1.13', 'human toxicity', 'HTPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'ionising radiation', 'IRP_HE'),
    ('ReCiPe Midpoint (H) V1.13', 'marine ecotoxicity', 'METPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'marine eutrophication', 'MEP'),
    ('ReCiPe Midpoint (H) V1.13', 'metal depletion', 'MDP'),
    ('ReCiPe Midpoint (H) V1.13', 'natural land transformation', 'NLTP'),
    ('ReCiPe Midpoint (H) V1.13', 'ozone depletion', 'ODPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'particulate matter formation', 'PMFP'),
    ('ReCiPe Midpoint (H) V1.13', 'photochemical oxidant formation', 'POFP'),
    ('ReCiPe Midpoint (H) V1.13', 'terrestrial acidification', 'TAP100'),
    ('ReCiPe Midpoint (H) V1.13', 'terrestrial ecotoxicity', 'TETPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'urban land occupation', 'ULOP'),
    ('ReCiPe Midpoint (H) V1.13', 'water depletion', 'WDP')
]

In [63]:
midpoints = [m[1] for m in methods]
scores = np.zeros((2, len(methods)))
lcas = [
    bw.LCA({actPV:1}),
    bw.LCA({actWind:1})
    ]


for i, lca in enumerate(lcas):
    lca.lci()
    for j, m  in enumerate(methods):
        lca.switch_method(m)
        lca.lcia()
        scores[i,j] = lca.score

In [64]:
lciaScores = xr.DataArray(scores, coords=[["PV", "Wind"], midpoints], dims=["Tech", "midpoint"])

In [65]:
lciaScores

<xarray.DataArray (Tech: 2, midpoint: 18)>
array([[ 2.55908547e-03,  2.99482003e-02,  9.70316326e-03,
         1.04899593e-02,  1.96835748e-05,  3.05531485e-02,
         7.11322857e-03,  9.23954536e-03,  1.88489337e-05,
         1.68457718e-02, -5.76878100e-06,  3.63754566e-09,
         7.28641280e-05,  1.44271468e-04,  1.60939713e-04,
         3.60146608e-05,  7.22719511e-03,  7.01587020e-04],
       [ 6.05562491e-04,  1.55864385e-02,  5.15925984e-03,
         5.90710735e-03,  7.07948168e-06,  1.05841837e-02,
         2.02218641e-03,  5.15185752e-03,  5.79419794e-06,
         8.77965280e-03, -3.73671042e-06,  1.14287113e-09,
         4.59953692e-05,  7.76168287e-05,  6.02776517e-05,
         5.52239703e-06,  1.66344951e-03,  1.90545755e-04]])
Coordinates:
  * Tech      (Tech) <U4 'PV' 'Wind'
  * midpoint  (midpoint) <U31 'agricultural land occupation' ... 'water deple...

Now get the monetization factors.

In [66]:
monetizationFactors = pd.read_excel("arendt-et-al.xlsx")
monetizationFactors

,Method,Impact category,unit,min,central,max,note
0,Stepwise,Global warming,2019€/kg CO2-eq,NaN,0.107282,NaN,NaN
1,Stepwise,Acidification,2019€/kg SO2-eq,NaN,0.19388,NaN,NaN
2,Stepwise,Stratospheric ozone depletion,2019€/kg CFC-11-eq,NaN,129.25506,NaN,NaN
3,Stepwise,Freshwater eutrophication,2019€/PO4-eq,NaN,1.551061,NaN,NaN
4,Stepwise,Marine eutrophication,2019€/kg NO3-eq,NaN,0.572,NaN,NaN
...,...,...,...,...,...,...,...
157,EVR,Particulate matter,2019€/kg PM 2.5-eq,NaN,36.038,NaN,NaN
158,EVR,Land use,2019€/m2/yr,0.0000,NaN,6.170,"min: natural forest, max: heterogeneous agricu..."
159,EVR,Land transformation,2019€/m2,0.1544,NaN,7.720,"min: countries with low biodiversity, max: cou..."
160,EVR,Soil organic matter,2019€/kg C-eq,NaN,NaN,NaN,NaN


Let us just work with Stepwise for now.

In [107]:
stepwiseFactors = monetizationFactors[monetizationFactors["Method"] == "Stepwise"]
stepwiseFactors = xr.DataArray(np.array(stepwiseFactors["central"]), coords=[stepwiseFactors["Impact category"]], dims=["Impacts"])
stepwiseFactors = stepwiseFactors.astype(float)

Define conversion factors between ReCiPe midpoints and Stepwise categories.

In [94]:
convFactors = xr.DataArray(
    np.zeros((18, 18)), 
    coords=[stepwiseFactors.coords["Impacts"], midpoints],
    dims = ["Impacts", "midpoint"]
    )

In [95]:
convFactors.loc["Global warming", "climate change"] = 1
convFactors.loc["Water use", "water depletion"] = 1
convFactors.loc["Stratospheric ozone depletion", "ozone depletion"] = 1
convFactors.loc["Acidification", "terrestrial acidification"] = 1
convFactors.loc["Terrestrial ecotoxicity", "terrestrial ecotoxicity"] = 1
convFactors.loc["Human toxicity", "human toxicity"] = 1.81e-07 * 11.5
convFactors.loc["Freshwater eutrophication", "freshwater eutrophication"] = 3.03
convFactors.loc["Marine eutrophication", "marine eutrophication"] = 4.34
convFactors.loc["Photochemical oxidation", "photochemical oxidant formation"] = 0.496
convFactors.loc["Mineral resources", "metal depletion"] = 1 # couldn't find a conversion
convFactors.loc["Fossil resources", "fossil depletion"] = 44.5 # heating value of oil in MJ/kg
convFactors.loc["Ionizing radiation", "ionising radiation"] = 1
convFactors.loc["Freshwater ecotoxicity", "freshwater ecotoxicity"] = 1
convFactors.loc["Marine ecotoxicity", "marine ecotoxicity"] = 1
convFactors.loc["Particulate matter", "particulate matter formation"] = 0.28
convFactors.loc["Land use", "agricultural land occupation"] = 1
convFactors.loc["Land use", "urban land occupation"] = 1
convFactors.loc["Land transformation", "natural land transformation"] = 1

In [96]:
convFactors

<xarray.DataArray (Impacts: 18, midpoint: 18)>
array([[0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.0300e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 4.3400e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00],
...
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 2.8000e-01, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00],
       [1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 1.0000e+00]])
Coordinates:
  * Impacts   (Impacts) object 'Global warming' 'Acidification' ... 'Water use'
  * midpoint  (midpoint) <U31 'agricultural land occupation' ... 'water deple...

In [108]:
impacts = convFactors.dot(lciaScores)
impacts

<xarray.DataArray (Impacts: 18, Tech: 2)>
array([[ 2.99482003e-02,  1.55864385e-02],
       [ 1.60939713e-04,  6.02776517e-05],
       [ 3.63754566e-09,  1.14287113e-09],
       [ 5.96412317e-05,  2.14508295e-05],
       [ 8.18043722e-05,  2.51468191e-05],
       [ 7.15586481e-05,  3.84979470e-05],
       [ 1.68457718e-02,  8.77965280e-03],
       [ 4.31790765e-01,  2.29587063e-01],
       [ 6.35963786e-08,  2.20309784e-08],
       [ 7.11322857e-03,  2.02218641e-03],
       [ 1.04899593e-02,  5.90710735e-03],
       [ 3.60146608e-05,  5.52239703e-06],
       [ 9.23954536e-03,  5.15185752e-03],
       [ 2.04019558e-05,  1.28787034e-05],
       [ 9.78628058e-03,  2.26901200e-03],
       [-5.76878100e-06, -3.73671042e-06],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 7.01587020e-04,  1.90545755e-04]])
Coordinates:
  * Impacts  (Impacts) object 'Global warming' 'Acidification' ... 'Water use'
  * Tech     (Tech) <U4 'PV' 'Wind'

In [110]:
stepwiseFactors * impacts

<xarray.DataArray (Impacts: 18, Tech: 2)>
array([[3.21289375e-03, 1.67213958e-03],
       [3.12029916e-05, 1.16866311e-05],
       [4.70171183e-07, 1.47721877e-07],
       [9.25071705e-05, 3.32715386e-05],
       [4.67921009e-05, 1.43839805e-05],
       [5.17961746e-05, 2.78659036e-05],
       [           nan,            nan],
       [2.23235826e-03, 1.18696511e-03],
       [1.24282714e-03, 4.30538632e-04],
       [1.77830714e-05, 5.05546603e-06],
       [2.48753230e-07, 1.40077954e-07],
       [1.65269431e-05, 2.53419967e-06],
       [           nan,            nan],
       [1.79319808e-03, 1.13195354e-03],
       [1.51785212e-03, 3.51923762e-04],
       [           nan,            nan],
       [           nan,            nan],
       [           nan,            nan]])
Coordinates:
  * Impacts  (Impacts) object 'Global warming' 'Acidification' ... 'Water use'
  * Tech     (Tech) <U4 'PV' 'Wind'